# 从 Boosted Tree 到 GBDT 再到 XGBoost

## Table Of Content

简单带过 Boosting , BT, GBDT, 详细推导 XGBoost.

XGBoost 是非常流行的工具,在 GBDT 的基础上做了很多优化, 在各种比赛和工业界都广泛应用.

- Boosting
- Boosted Tree
- Gradient Boosted Tree
    - 参考
- XGBoost
    - Loss项 + 泰勒展开
    - 正则项
    - 结构分数
    - 树结构的搜索算法
    - 参考
- 其它 

## Boosting 

 Bagging 和 Boosting 都是 Ensemble Learing 集成学习的思想, 分别对应了并行和串行的弱学习器的集成. 
 
 Boosting Tree 提升树, 则是以决策树为弱学习器的一类 Boosting方法. 而这里的决策树其实是 Regression Tree 回归树.
 
 一般说到 Boosted Tree 都是用 MSE (Mean Square Error) 作为损失函数来优化由多棵回归树组成的集成学习器. 而推广到其它 Loss Function, 则就是 Gradient Boosted Tree, 或称为 GBDT (Gradient Boosted Decision Tree).

有时候也会提到 MART (Multiple Additive Regression Tree), 其实也是同一个东西, 因为 Boosted Tree 就是利用 加法模型(基函数的线性组合) 和 向前分布算法, 以回归树为基函数的集成方法.

## Boosted Tree

先来简单说下普通 Bossted Tree 的算法. **Bossted Tree 是迭代多棵回归树来共同决策的, 最后的结果是由训练出的多棵树的结果相加而得到的.** 
那么怎么学习一个新的树呢? 

答案是: 每一棵树都是通过 前面所有树的结果 和 残差 (Residual), 来拟合一个当前残差的回归树.

下图训练一个提升树模型来预测年龄:
训练集是4个人, A, B, C, D年龄分别是14，16，24，26. 样本中有购物金额, 上网时长, 经常到百度知道提问等特征. 新树的建立过程如下:

<img src="images/boosted_tree_example.jpg" width="600" />

通过这个简单的例子可以看到, 从第一棵树计算出的残差, 被用来作为下一棵树的输入, 并生成出拟合残差的新树. 通过这样的方式, 使得多棵树的值加起来就可以逼近样本的真实值. 以下是 Boosted Tree 的算法描述:

1. 初始化 $f_0(x)=0$
2. 对  $m = 1,2,3,...,M$ # M棵树
    - 计算残差 $r_{mi}=y_i - f_{m-1}(x_i), i=1,2,...,N$ # N个样本
    - 用下一棵树来拟合残差 $r_{mi}$, 得到 $T(x;\theta m)$
    - 更新 $f_m(x)=f_{m-1}(x)+T(x;\theta m)$
3. 最终得到提升树
    - $f_M(x)=\sum_{m=1}^M T(x;\theta m)$

通常只有 MSE 作为损失函数的时候, 才会用残差来作为下一棵树的输入进行拟合. 那么其它的损失函数呢? 由此可以引出更通用的 Gradient Booted Tree.

## Gradient Boosted Tree

Gradient Boosted Tree 也就是 GBDT, 是利用损失函数在当前模型下的梯度值, 来逐步逼近最优解. 类似于通常梯度下降法, 在参数空间中寻找局部最优解, 而 GBDT, 则是在函数空间中, 寻找最优的集成后的函数, 也就是最终的集成的模型.

<img src="images/boosted_tree_gradient_algorithm.png" width="600"/>

1. 初始化, 这里计算的是一个常数值, 使得损失函数 $L$ 最小化
2. (a) 计算损失函数 $L$ 在当前模型下负梯度的值, 作为残差的估计
2. (b) 生成拟合残差近似值的回归树, 得到样本在叶结点的集合(哪些样本属于哪些叶结点)
2. (c) 计算叶结点的分数
2. (d) 集成模型
3. 得到最终模型

<img src="images/boosted_tree_gradient.png" width="600"/>

由上图可见, 其它损失函数的负梯度的值; 不难看出, 提升树所用的残差 $y_i - f(x_i)$ 就是 $MES: \frac{1}{2}[y_i-f(x_i)]^2$ 对于 $f(x_i)$ 的负梯度的值. 在我理解来看, Boosted Tree 就是 GBDT 以 MSE 作为损失函数时的特殊情况.

在有的推导中, 貌似是用 GBDT 解分类问题时, 还有一种推导, 会用牛顿法算出在负梯度方向上的步长, 进而继续优化. 通常的 GBDT 只用到了目标函数的一阶导数的信息. 通常有人说 GBDT 和 XGBoost 的区别时会说到, GBDT 只用一阶信息, 而 XGBoost 用了二阶信息. 而使用牛顿法求步长的梯度提升树应该同样也用到了二阶的信息.

#### 参考

- << 统计学习方法 p.136 >>
- GBDT：梯度提升决策树 https://www.jianshu.com/p/005a4e6ac775
- GBDT（MART） 迭代决策树入门教程 | 简介 http://blog.csdn.net/suranxu007/article/details/49910323
- Wiki https://en.wikipedia.org/wiki/Gradient_boosting#Gradient_tree_boosting
- 当我们在谈论GBDT：Gradient Boosting 用于分类与回归 https://zhuanlan.zhihu.com/p/25257856?refer=data-miner

## XGBoost

XGBoost 是 Extreme Gradient Boosting 的缩写, XGBoost 的推导从最基本的 [ 目标函数 = 损失 + 正则 ] 一点一点的推导出非常具有扩展性的算法, 而且很自然的引出了, 划分叶结点的计算方法, 还有预剪枝的功能.

接下来就着重推导一下 XGBoost 的公式 : <font color=#FF7F50 size=3>(以下公式预警)</font>

作为解决监督学习的问题的方法, 我们从最基本的目标函数开始. Objective Function : Training Loss + Regularization

$$obj(\theta) = L(\theta) + \Omega(\theta)$$

Training Loss 控制了模型的拟合程度, Loss 自然是越小越好; 而 Regularization 控制了模型的复杂程度. 通常的原则是, 我们希望得到一个即简单的,又有更高预测性的模型. 这里就涉及到了 bias-variance 的权衡.

以上原则适用于几乎所有监督学习, 在 Boosting Tree 中的话, 就是如下:

$$\hat{y_i} = \sum_{k=1}^K f_k(x_i), f_k\in F$$
$$obj(\theta)=\sum_i^N l(y_i, \hat{y_i}) + \sum_{k=1}^K \Omega(f_k)$$ 

- $K$: 树的数量
- $N$: 样本数
- $f$: 一棵树的模型函数
- $F$: 是函数空间, 所有可能回归树弱分类器的集合
- $l$: 损失函数

这里有一点需要再提一下, 通常求解最优化问题都是在参数空间里面找到使目标函数最小的最优参数; 而在梯度提升树里, 最优化的方向是在函数空间里 (所有可能的回归树里) 找到最优的 $f$

回顾之前所说的, Bossted Tree 是迭代多棵回归树来共同决策的, 最后的结果是由训练出的多棵树的结果相加而得到的. 因为是加法模型, 所以我们如果可以分而治之的求得每一次迭代的最优树, 那么整体来看我们的模型也是最优的.

假设初始情况第一棵决策树桩的值是 $\hat{y_i}^{(0)} = 0$, 每一次迭代是在之前的基础上加上新的模型:

\begin{aligned}
    \hat{y_i}^{(0)} &= 0 \\
    \hat{y_i}^{(1)} &= f_1(x_i) &= \hat{y_i}^{(0)} + f_1(x_i) \\
    \hat{y_i}^{(2)} &= f_1(x_i) + f_2(x_i) &= \hat{y_i}^{(1)} + f_2(x_i) \\
    &... \\
    \hat{y_i}^{(t)} &= \sum_{k=1}^t f_k(x_i) &= \hat{y_i}^{(t-1)} + f_t(x_i) \\
\end{aligned}

第$t$轮迭代时, 我们这一步的目标函数就是

\begin{aligned}
    obj(\theta)^{(t)} &= \sum_i^N l(y_i, \hat{y_i}^{(t)}) + \sum_{k=1}^K \Omega(f_k) \\
                      &= \sum_i^N l(y_i, \hat{y_i}^{(t-1)} + f_y(x_i)) + \Omega(f_t) + constant  \text{ }\text{ (Eq.1)}
\end{aligned}

XGBoost 的其中一个扩展性, 就是支持不同的 loss funciton, 这里如果使用 MSE 的话, 那么

\begin{aligned}
    obj(\theta)^{(t)} &= \sum_i^N (y_i - (\hat{y_i}^{(t-1)} + f_y(x_i)))^2 + \Omega(f_t) + constant \\
                      &= \sum_i^N [2( \hat{y_i}^{(t-1)}  - y_i)f_i(x_i) + f_t(x_i)^2] + \Omega(f_t) + constant \text{ }\text{ (Eq.2)}\\
\end{aligned}

可见 MSE 是比较简单的, 最终得到以残差为参数的一次项和一个二次项. 那么其它的损失函数怎么办呢? 

答案是 **泰勒展开**

### 泰勒展开

XGBoost 使用的泰勒展开到2次的多项式来近似求解其它的损失函数. 因此也要求使用的损失函数是可以对 $f(x)$ 进行一阶和二阶求导的. 

$$\text{泰勒二次展开: } f(x+\Delta x) \simeq f(x) + f'(x)\Delta x + \frac{1}{2}f''(x)\Delta x^2$$

以下是通用的泰勒二次展开后的损失函数.


\begin{aligned}
    obj(\theta)^{(t)} &\simeq \sum_i^N [ l(y_i, \hat{y_i}^{(t-1)}) + g_if_t(x_i) + \frac{1}{2}h_if_t^2(x_i) ] + \Omega(f_t) + constant 
     \\
     g_i &= \frac{\partial l(y_i, \hat{y_i}^{(t-1)})}{\partial(\hat{y_i}^{(t-1)})} \\
     h_i &= \frac{\partial^2 l(y_i, \hat{y_i}^{(t-1)})}{\partial(\hat{y_i}^{(t-1)})}
\end{aligned}

$g_i, h_i$ 分别是损失函数的一阶和二阶导数; 去掉不影响最优化的常数项, 就得到在每一次迭代的时候, 通用的目标函数 (Eq.3)

$$obj(\theta)^{(t)} \simeq \sum_i^N [ g_if_t(x_i) + \frac{1}{2}h_if_t^2(x_i) ] + \Omega(f_t)   \text{ }\text{ (Eq.3)}$$


> Tips:
> 如果觉得 Eq.3 不太好理解, 那么请对照 MSE 均方误差作为损失函数的例子来看, 那么 $g_i, h_i$ 就分别是
>
> $g_i = \frac{\partial (y_i - \hat{y_i}^{(t-1)})^2}{\partial(\hat{y_i}^{(t-1)})} = 2( \hat{y_i}^{(t-1)} - y_i)$

> $h_i = \frac{\partial^2 (y_i - \hat{y_i}^{(t-1)})^2}{\partial(\hat{y_i}^{(t-1)})}$ = 2

### 正则项  Regularization

接下来看目标函数的第二部分正则项, 正则项控制了模型的复杂程度, 也就是树的复杂度, 对此我们需要首先定义一下树 $f_t(x)$, 然后定义树的复杂度 $\Omega(f_t)$

$$f_t(x) = w_{q(x)}$$

$q(x)$ 把样本划分到 $T$ 个叶结点中, 而 $w_{q(x)}$ 就是每个叶结点的分数,一棵树就被抽象为 $f_t(x)$

$$w \in R^T$$
$$q:R^d \to {1,2,...,T}$$

<img src="images/boosted_tree_complex1.png" width="600"/>

然后, 在 XGBoot 中, 我们定义复杂度为

$$\Omega(f) = \gamma T + \frac{1}{2}\lambda\sum_{j=1}^T w_j^2$$
$$T是叶结点的个数, 后面的是叶结点分数的 L2 范数$$

当然树的复杂度可以有多种定义方式, 作者所用上面这种, 在实践中被证明很有用. 传统决策树进行复杂度控制的方法, 都是启发式的剪枝算法, 而在 XGBoost 中通过严格的定义, 我们能更清楚的知道模型学习的方向, 并且在实践中被证明是很有效的.

<img src="images/boosted_tree_complex2.png" width="650"/>

### 损失函数 / 结构分数 ( Structure Score )

现在重写我们的第 $t$ 棵树的目标函数 Eq.3, 定义叶结点 $j$ 上包含样本的集合 $I_j = \{i | q(x_i)=j\}$, $f_t(x_i)=w_{q(x_i)}$

\begin{aligned}
    obj(\theta)^{(t)} &\simeq \sum_i^N [ g_if_t(x_i) + \frac{1}{2}h_if_t^2(x_i) ] + \Omega(f_t)   \text{ }\text{ (Eq.3)} \\
                      &=\sum_i^N [ g_if_t(x_i) + \frac{1}{2}h_if_t^2(x_i)] + \gamma T + \frac{1}{2}\lambda\sum_{j=1}^T w_j^2 \\
                      &=\sum_i^N [ g_i w_{q(x_i)} + \frac{1}{2}h_i w^2_{q(x_i)}] + \gamma T + \frac{1}{2}\lambda\sum_{j=1}^T w_j^2 \\
                      &=\sum_j^T [ (\sum_{i\in I_j}g_i) w_j + \frac{1}{2} (\sum_{i\in I_j}h_i+\lambda ) w^2_j
                      ] + \gamma T  \text{ }\text{ (Eq.4)}\\
\end{aligned}

从 Eq.4 看出, 因为同一个叶结点上的样本得到相同的分数, 所以按每个叶结点合并以后, 目标函数变成了 $T$ 个叶结点上 $w_j$ 的二次函数的加和. 二次函数是凸函数, 可以用梯度方法求出使目标函数最小的最优的参数 $w_j$

> **也就是对于一个已定结构的树, 我们可以用最优化的方法求出叶结点上的最佳分数; 进一步来想, 如果我们遍历一棵树的所有可能的结构, 求出每一种结构下最优的 $w_j$, 那么我们也就能求出每一种结构对应的目标函数的值 (每一种树结构的结构分数), 最优的树不就找到了么**
>
> **当然实际上会有更有效的方法来替代遍历所有结构这一步, 但总体思想脉络就是这样.**

以下推导最优$w_j$和结构范数

$$\text{令 } G_j = \sum_{i\in I_j}g_i, \text{ } H_j=\sum_{i\in I_j}h_i$$
$$\text{则有 } obj(\theta)^{(t)} = \sum_j^T [ G_j w_j + \frac{1}{2} ( H_j +\lambda ) w^2_j
                      ] + \gamma T  \text{ }\text{ (Eq.5)}$$

因为上式中 $w_j$ 都互相独立, 所以

$$G_j + (H_j + \lambda) w_j^* = 0$$

$$w_j^* = - \frac{G_j}{H_j + \lambda}$$

$$obj^* = -\frac{1}{2}\sum_j^T \frac{G_j^2}{H_j+\lambda} + \gamma T \text{ }\text{ (Eq.6)}$$

Eq.6 代表了一个树结构 $q(x)$ 有多好, 值越小, 结构越好

<img src="images/boosted_tree_structure_score.png" width="650"/>

### 单棵树结构的搜索算法 Searching Algorithm

如上文所说, 最简单的方法就是遍历所有可能的树结构, 通过 Eq.6 求出最优的结构分数; 然而实际中很困难. 我们用的是贪心法(Greedy Learning), 从树桩开始, 每次在已有叶结点上新开一个分支. *划分之前* 和 *划分之后* 的两种结构就也可以用结构分数来进行比较. 类似于 `CART` 的基尼指数, 这里也可有得到一个 `Gain` 来决定是否划分结点.

$$Gain = \frac{1}{2} [  \frac{G_L^2}{H_L+\lambda} + \frac{G_R^2}{H_R+\lambda} - \frac{(G_L+G_R)^2}{H_L+H_R+\lambda} ] - \gamma $$

$$Gain = [ 划分后的左枝分数 + 划分后的右枝分数 - 划分前的分数 ] - 新增一个结点的复杂度代价$$

在一个现有结点上如何划分的算法如下:

- 遍历所有属性
    - 按属性值从小到大依次排序
    - 从左到右, 对有序列尝试分割
    - 贪婪的找出 Gain 最大的分割


<img src="images/boosted_tree_split.png" width="600"/>

从上面的公式可以看出, 引入分割不一定更好, 如果一次分割的 `Gain` 小于 新增一个节点的复杂度代价的话, 这个分割一定是会被舍弃的, 这就相当于传统树模型中恩预剪枝处理. 

> 如作者陈天奇所说, 正式的推导带有正则项的目标函数, 像计算结构分数和剪枝这样的策略都会自然而然的出现, 不再是一种启发式的操作了

自此, XGBoost 的多树加和的集成模型, 和每一轮迭代, 每一棵树的结构的生成和其叶结点分数就都推导出来了.

这一部分, 介绍了如何通过目标函数优化的方法, 比较严格地推导出 boosted tree的学习, 类似的思想在解决机器学习的问题中其实是十分正确且通用的.

#### 参考

- XGBoost 与 Boosted Tree by 陈天奇 http://www.52cs.org/?p=429
- Introduction to Boosted Trees, Tianqi Chen , 2014 http://homes.cs.washington.edu/~tqchen/pdf/BoostedTree.pdf
- http://xgboost.readthedocs.io/en/latest/model.html#additive-training

### 其它
- 机器学习算法中GBDT和XGBOOST的区别有哪些？https://www.zhihu.com/question/41354392
- How to Configure the Gradient Boosting Algorithm https://machinelearningmastery.com/configure-gradient-boosting-algorithm/

= THE END =